Let's define the functions we need to solve problems 7 - 10.

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import pandas as pd

def linear_regr_func_regularized(y, x, lam, size1):
    I = np.zeros([size1, size1])
    np.fill_diagonal(I, 1.0)
    X_T = np.transpose(x)
    X_inv = np.linalg.inv( np.dot(X_T, x) + lam*I )
    X_ps_inv = np.dot(X_inv, X_T)
    W_lin = np.dot(X_ps_inv, y)
    return W_lin

def prepare_data_num_vs_all(train_data, test_data, num):
    y_train = np.zeros(len(train_data))
    y_test = np.zeros(len(test_data))
    for i in range(len(train_data)):
        if(train_data[i,0]==num):
            y_train[i]= 1
        else:
            y_train[i]= -1
            
    for i in range(len(test_data)):
        if(test_data[i,0]==num):
            y_test[i]= 1
        else:
            y_test[i]= -1
    return y_train, y_test

def prepare_data_num_vs_num(train_data, test_data, num1, num2):
    train_2_nums = train_data[(train_data[:, 0]== num1) | (train_data[:, 0]== num2)]
    test_2_nums = test_data[(test_data[:, 0]== num1) | (test_data[:, 0]== num2)]
    
    y_train = np.zeros(len(train_2_nums))
    y_test = np.zeros(len(test_2_nums))
    for i in range(len(train_2_nums)):
        if(train_2_nums[i,0]==num2):
            y_train[i]= 1
        else:
            y_train[i]= -1
            
    for i in range(len(test_2_nums)):
        if(test_2_nums[i,0]==num2):
            y_test[i]= 1
        else:
            y_test[i]= -1
    return train_2_nums, y_train, test_2_nums, y_test 

def error_in_out_evaluation(f_labels, g_labels):
    N_points = len(f_labels)
    E_in = 0.0
    for i in range(0, N_points):
        if(f_labels[i] != g_labels[i]):
            E_in+=1
            
    E_in = E_in / N_points
    return E_in

def prepare_data(data):
    E = np.zeros([len(data)])
    E.fill(1)
    prepared = np.stack([E, data[:,1], data[:, 2]], axis =1)
    return prepared

def feat_transform(data):
    E = np.zeros([len(data)])
    E.fill(1)
    prepared = np.stack([E, data[:,1], data[:, 2], data[:,1]*data[:, 2], (data[:,1])**2, (data[:, 2])**2 ], axis =1)
    return prepared

<b>Problem 7</b>

In [25]:
train_set = pd.read_table("features.train", delim_whitespace=True, header=None).values
test_set = pd.read_table("features.test", delim_whitespace=True, header=None).values

#problem 7
lam = 1.0
prep_train = prepare_data(train_set)
E_in_7 = np.zeros([5])
counter =0
nums_list_7 = (5, 6, 7, 8, 9)
for i in nums_list_7:
    y_train, y_test = prepare_data_num_vs_all(train_set, test_set, i)
    w_reg = linear_regr_func_regularized(y_train, prep_train, lam, 3)
    g_labels_in = np.sign(np.dot(w_reg, np.transpose(prep_train) ))
    E_in_7[counter] = error_in_out_evaluation(y_train, g_labels_in)
    counter+=1
print("min E_in:", nums_list_7[np.argmin(E_in_7)], "vs all")

min E_in: 8 vs all


So the correct answer is <b>[d]</b>.

<b>Problem 8</b>

In [26]:
E_out_8 = np.zeros([5])
counter =0
nums_list_8 = (0, 1, 2, 3, 4)
for i in nums_list_8:
    y_train, y_test = prepare_data_num_vs_all(train_set, test_set, i)
    nonlin_feat_in = feat_transform(train_set)
    nonlin_feat_out = feat_transform(test_set)
    w_reg = linear_regr_func_regularized(y_train, nonlin_feat_in, lam, 6)
    g_labels_out = np.sign(np.dot(w_reg, np.transpose(nonlin_feat_out) ))
    E_out_8[counter] = error_in_out_evaluation(y_test, g_labels_out)
    counter+=1
print("min E_out:", nums_list_8[np.argmin(E_out_8)], "vs all")

min E_out: 1 vs all


The correct answer is <b>[b]</b>.

<b>Problem 9</b>

In [29]:
E_in_9 = np.zeros([10])
E_out_9 = np.zeros([10])
E_in_9_nonlin = np.zeros([10])
E_out_9_nonlin = np.zeros([10])
prep_train = prepare_data(train_set)
prep_test = prepare_data(test_set)
counter =0
nums_list_9 = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
for i in nums_list_9:
    y_train, y_test = prepare_data_num_vs_all(train_set, test_set, i)

    w_reg = linear_regr_func_regularized(y_train, prep_train, lam, 3)
    g_labels_in = np.sign(np.dot(w_reg, np.transpose(prep_train) ))    
    g_labels_out = np.sign(np.dot(w_reg, np.transpose(prep_test) ))
    E_in_9[counter] = error_in_out_evaluation(y_train, g_labels_in)
    E_out_9[counter] = error_in_out_evaluation(y_test, g_labels_out)
    
    nonlin_feat_in = feat_transform(train_set)
    nonlin_feat_out = feat_transform(test_set)
    w_reg_nonl = linear_regr_func_regularized(y_train, nonlin_feat_in, lam, 6)
    g_labels_in_nonl = np.sign(np.dot(w_reg_nonl, np.transpose(nonlin_feat_in) ))
    g_labels_out_nonl = np.sign(np.dot(w_reg_nonl, np.transpose(nonlin_feat_out) ))
    E_in_9_nonlin[counter] = error_in_out_evaluation(y_train, g_labels_in_nonl)
    E_out_9_nonlin[counter] = error_in_out_evaluation(y_test, g_labels_out_nonl)
    counter+=1
results = np.zeros([5], dtype=bool)
results[0] = np.all(E_in_9_nonlin > E_out_9_nonlin)
results[1] = np.all(0.95* E_out_9 >= E_out_9_nonlin)
results[2] = np.all(E_out_9 == E_out_9_nonlin)
results[3] = np.all( E_out_9 <= 0.95*E_out_9_nonlin)
results[4] = (E_out_9[5] > E_out_9_nonlin[5])and (0.95*E_out_9[5] < E_out_9_nonlin[5])
print(results)

[False False False False  True]


The correct answer is <b>[e]</b>.

<b>Problem 10</b>

In [35]:
x_train_1_5, y_train_1_5, x_test_1_5, y_test_1_5 =prepare_data_num_vs_num(train_set, test_set, 1, 5)
E_in_10 = np.zeros([2])
E_out_10 = np.zeros([2])
E_in_10_nonlin = np.zeros([2])
E_out_10_nonlin = np.zeros([2])
prep_train = prepare_data(x_train_1_5)
prep_test = prepare_data(x_test_1_5)
counter =0
lambda_list = (0.01, 1)
for i in lambda_list:
    w_reg = linear_regr_func_regularized(y_train_1_5, prep_train, i, 3)
    g_labels_in = np.sign(np.dot(w_reg, np.transpose(prep_train) ))    
    g_labels_out = np.sign(np.dot(w_reg, np.transpose(prep_test) ))
    E_in_10[counter] = error_in_out_evaluation(y_train_1_5, g_labels_in)
    E_out_10[counter] = error_in_out_evaluation(y_test_1_5, g_labels_out)
    
    nonlin_feat_in = feat_transform(x_train_1_5)
    nonlin_feat_out = feat_transform(x_test_1_5)
    w_reg_nonl = linear_regr_func_regularized(y_train_1_5, nonlin_feat_in, i, 6)
    g_labels_in_nonl = np.sign(np.dot(w_reg_nonl, np.transpose(nonlin_feat_in) ))
    g_labels_out_nonl = np.sign(np.dot(w_reg_nonl, np.transpose(nonlin_feat_out) ))
    E_in_10_nonlin[counter] = error_in_out_evaluation(y_train_1_5, g_labels_in_nonl)
    E_out_10_nonlin[counter] = error_in_out_evaluation(y_test_1_5, g_labels_out_nonl)
    counter+=1
results = np.zeros([5], dtype=bool)
results[0] = np.all(( (E_in_10< E_out_10), (E_in_10_nonlin < E_out_10_nonlin) ))
results[1] = np.all(E_in_10 == E_in_10_nonlin)
results[2] = np.all(E_out_10 == E_out_10_nonlin)
results[3] = np.all (( ( E_in_10[0] < E_in_10[1]), ( E_out_10[0] < E_out_10[1]), (E_in_10_nonlin[0] < E_in_10_nonlin[1]), ( E_out_10_nonlin[0] < E_out_10_nonlin[1]) ))
results[4] = np.all (( ( E_in_10[0] > E_in_10[1]), ( E_out_10[0] > E_out_10[1]), (E_in_10_nonlin[0] > E_in_10_nonlin[1]), ( E_out_10_nonlin[0] > E_out_10_nonlin[1]) ))
print(results)

[ True False False False False]


The correct answer is <b>[a]</b>.